In [ ]:
%matplotlib inline
import flopy
import matplotlib.pyplot as plt
import matplotlib.tri as tri
import numpy as np
import pathlib as pl
import xarray as xa

In [ ]:
fpth = pl.Path("../background/newMVTruth.nc")

In [ ]:
nc_ds = xa.open_dataset(fpth)

In [ ]:
nc_ds

In [ ]:
name = "mv"
ws = pl.Path("mv")

## Define discretization

In [ ]:
top = nc_ds["top_layer1"].values + 2
top[nc_ds["lake_location"].values == 1] = 11

In [ ]:
nlay = 5
nrow, ncol = top.shape
nlay, nrow, ncol

In [ ]:
shape3d = (nlay, nrow, ncol)
shape2d = (nrow, ncol)

In [ ]:
delr = delc = 500.

In [ ]:
botm = [
    nc_ds["bottom_layer1"].values,
    nc_ds["bottom_layer2"].values,
    nc_ds["bottom_layer3"].values,
    nc_ds["bottom_layer4"].values,
    nc_ds["bottom_layer5"].values,
]

In [ ]:
lake_location = nc_ds["lake_location"].values

In [ ]:
nper = 4
tdis_ds = [
    (1.0, 1, 1),
    (1.0, 1, 1),
    (1.0, 1, 1),
    (1.0, 1, 1),
]

In [ ]:
# make passthrough for layer 3 where the clay is absent
idomain = np.ones((nlay, nrow, ncol))
idomain[2,:,:] = nc_ds.clay_location.data
idomain[idomain==0] = -1
idomain = idomain.astype(int)

## Define starting heads

In [ ]:
strt = [
    nc_ds["head_layer1"].values,
    nc_ds["head_layer2"].values,
    nc_ds["head_layer3"].values,
    nc_ds["head_layer4"].values,
    nc_ds["head_layer5"].values,
]

## Define aquifer properties 

In [ ]:
kvkh_ratio = 0.4

In [ ]:
lake_location = nc_ds["lake_location"].to_numpy()

In [ ]:
clay_location = nc_ds["clay_location"].to_numpy()

In [ ]:
k = np.array(
    [nc_ds[f'k1_layer{i+1}'].data for i in range(5)])
k33 = k.copy()*kvkh_ratio

In [ ]:
k1 = k[0, :, :]
k1[lake_location == 1] = 2000000.0
k[0, :, :] = k1

In [ ]:
k1 = k33[0, :, :]
k1[lake_location == 1] = 2000000.0
k33[0, :, :] = k1

## Define the recharge

In [ ]:
recharge = np.full(shape2d, 0.00365, dtype=float)
recharge[lake_location == 1] = 0.00205

## Define the river

In [ ]:
nriv = 18
stage = np.linspace(1.75, 0.05, nriv)
stage

In [ ]:
riv_bndname = ["PF", "PF"] + ["RIV" for i in range(nriv - 2)]

In [ ]:
river_spd = [(0, i + 22, 8, float(stage[i]), 1e5, float(stage[i]) - 1., riv_bndname[i]) for i in range(nriv)]
river_spd

## Define the wells

In [ ]:
well_spd = {
    1: [
        (4, 5, 14, -67000),
        (4, 34, 15, -268000),
    ],
    2: [
        (4, 5, 14, -67000),
        (4, 32, 5, -268000),
    ],
    3: [
        (4, 5, 14, -67000),
    ],
}

## Define the drain for the extra run

In [ ]:
drn_spd = {
    3: [
        (0, 22, 19, 2.0, 4e6)
    ],
}

## Define the head observations

In [ ]:
obs_rc_locs = [
    (2, 17),
    (3, 10),
    (6, 20),
    (12, 22),
    (14, 11),
    (16, 18),
    (17, 1),
    (18, 6),
    (19, 11),
    (18, 22),
    (26, 5),
    (27, 11),
    (28, 23),
    (30, 6),
    (33, 14),
    (36, 1),
    (37, 22),
]

In [ ]:
np.array(botm)[:, 3, 10]

In [ ]:
well_depth = [
    -200.74, -100.00, -100.00, -100.00, -224.62, 
    -100.00, -100.00, -100.00, -100.00, -100.00,
    -100.00, -100.00, -100.00, -100.00, -233.22,
    -100.00, -100.00,
]

In [ ]:
wt_obs = []
aq_layer = []
aq_obs = []
for idx, (i, j) in enumerate(obs_rc_locs):
    iloc = (i, j)
    tag = "head_layer1"
    wt_obs.append(float(nc_ds[tag].values[iloc]))
    wz = well_depth[idx]
    zcell = np.array(botm)[:, i, j]
    klay = 0
    for kk in range(1, nlay):
        z0 = zcell[kk - 1]
        z1 = zcell[kk]
        if wz < z0 and wz >= z1:
            klay = kk
            break
    tag = f"head_layer{klay + 1}"
    aq_layer.append(klay)
    aq_obs.append(float(nc_ds[tag].values[iloc]))

In [ ]:
print(wt_obs)

In [ ]:
print(aq_layer)

In [ ]:
print(aq_obs)

In [ ]:
cal_loc_wt = [(0, i, j) for i, j in obs_rc_locs]

In [ ]:
cal_loc_aq = [(aq_layer[idx], i, j) for idx, (i, j) in enumerate(obs_rc_locs)]

In [ ]:
obs_loc = {}
obs_loc["mv.gwf.wt.csv"] = [(f"WT{idx + 1: 02d}", "HEAD", (0, i, j)) for idx, (i, j) in enumerate(obs_rc_locs)]

In [ ]:
obs_loc["mv.gwf.aq.csv"] = [(f"AQ{idx + 1: 02d}", "HEAD", (aq_layer[idx], i, j)) for idx, (i, j) in enumerate(obs_rc_locs)]

In [ ]:
obs_loc["mv.gwf.scenario.csv"] = [
    ("Lake-N", "HEAD", (0, 3, 3)),
    ("Lake-S", "HEAD", (0, 15, 8)),
    ("Reilly", "HEAD", (4, 5, 14)),
    ("V1", "HEAD", (4, 34, 15)),
    ("V2", "HEAD", (4, 32, 5)),
    ("DW", "HEAD", (0, 22, 19)),
]


In [ ]:
obs_loc

## Build the model

In [ ]:
sim = flopy.mf6.MFSimulation(sim_name=name, sim_ws=ws)

In [ ]:
tdis = flopy.mf6.ModflowTdis(sim, nper=nper, perioddata=tdis_ds, time_units="DAY")

In [ ]:
ims = flopy.mf6.ModflowIms(sim, complexity="simple", outer_maximum=200, inner_maximum=100, outer_dvclose=1e-5, inner_dvclose=1e-6)

In [ ]:
gwf = flopy.mf6.ModflowGwf(sim, modelname=name, save_flows=True)

In [ ]:
dis = flopy.mf6.ModflowGwfdis(gwf, 
                              length_units="FEET", 
                              nlay=nlay, 
                              nrow=nrow, 
                              ncol=ncol, 
                              delr=delr,
                              delc=delc,
                              top=top,
                              botm=botm,
                              idomain=idomain
                             )

In [ ]:
ic = flopy.mf6.ModflowGwfic(gwf, strt=strt)

In [ ]:
npf = flopy.mf6.ModflowGwfnpf(gwf, save_specific_discharge=True, k=k, k33=k33)

In [ ]:
rch = flopy.mf6.ModflowGwfrcha(gwf, recharge=recharge)

In [ ]:
riv_obs = {"riv.obs.csv": [("PF", "RIV", "PF"), ("RIVFLOW", "RIV", "RIV")]}

In [ ]:
riv = flopy.mf6.ModflowGwfriv(gwf, boundnames=True, 
                              print_flows=True, 
                              maxbound=nriv, 
                              stress_period_data=river_spd, 
                              observations=riv_obs
                             )

In [ ]:
wel = flopy.mf6.ModflowGwfwel(gwf, stress_period_data=well_spd, maxbound=2, pname="pwell")

In [ ]:
drn = flopy.mf6.ModflowGwfdrn(gwf, stress_period_data=drn_spd, maxbound=1, pname="dewater")

In [ ]:
oc = flopy.mf6.ModflowGwfoc(
    gwf,
    head_filerecord=f"{name}.hds",
    budget_filerecord=f"{name}.cbc",
    saverecord=[("HEAD", "ALL"), ("BUDGET", "ALL")],
)

In [ ]:
gwf_obs = flopy.mf6.ModflowUtlobs(gwf, print_input=True, continuous=obs_loc)

In [ ]:
sim.write_simulation()

In [ ]:
sim.run_simulation()

## Plot the results

### Model Properties

In [ ]:
with flopy.plot.styles.USGSMap():
    fig, axs = plt.subplots(1, 5, figsize=(9, 3), sharey=True)
    fig.suptitle("Hydraulic conductivity")

    for idx in range(nlay):
        ax = axs[idx]
        mm = flopy.plot.PlotMapView(model=gwf, layer=idx, ax=ax)
        mm.plot_array(k[idx], masked_values=[2000000.0])
        ax.set_title(f"Layer {idx + 1}")

In [ ]:
with flopy.plot.styles.USGSMap():
    fig, axs = plt.subplots(1, 5, figsize=(9, 3), sharey=True)
    fig.suptitle("Bottom Elevation")

    for idx in range(nlay):
        ax = axs[idx]
        mm = flopy.plot.PlotMapView(model=gwf, layer=idx, ax=ax)
        mm.plot_array(botm[idx])
        ax.set_title(f"Layer {idx + 1}")

In [ ]:
with flopy.plot.styles.USGSMap():
    fig, axs = plt.subplots(1, 5, figsize=(9, 3), sharey=True)
    fig.suptitle("Cell thickness")
    z = gwf.modelgrid.cell_thickness

    for idx in range(nlay):
        ax = axs[idx]
        mm = flopy.plot.PlotMapView(model=gwf, layer=idx, ax=ax)
        mm.plot_array(z[idx])
        ax.set_title(f"Layer {idx + 1}")

### Simulated Heads and Drawdown

In [ ]:
levels = np.arange(2, 20., 2)

#### Calibration

In [ ]:
hds = gwf.output.head().get_data(totim=1.0)

In [ ]:
with flopy.plot.styles.USGSMap():
    fig, axs = plt.subplots(1, 5, figsize=(9, 3), sharey=True)
    fig.suptitle("Heads - calibration")

    for idx in range(nlay):
        ax = axs[idx]
        mm = flopy.plot.PlotMapView(model=gwf, layer=idx, ax=ax)
        mm.plot_array(hds)
        cs = mm.contour_array(hds, colors="white", levels=levels)
        plt.clabel(cs, inline=True, fontsize=8)
        ax.set_title(f"Layer {idx + 1}")

In [ ]:
v = gwf.output.budget().get_data(text="riv", totim=1.0)[0]["q"]
print(f"River infiltration: {np.all(v > 0)}\n{v}")

##### Calculate the residuals

In [ ]:
sim_wt = np.array([hds[idx] for idx in cal_loc_wt])

In [ ]:
resid_wt = sim_wt - np.array(wt_obs)
resid_wt

In [ ]:
sim_aq = np.array([hds[idx] for idx in cal_loc_aq])

In [ ]:
resid_aq = sim_wt - np.array(aq_obs)
resid_aq

In [ ]:
resid_gb = np.concatenate((resid_wt, resid_aq))

In [ ]:
print(f"Water Table Statistics\nMean Error: {resid_wt.mean()} ft.\nRMSE:       {np.sqrt((resid_wt**2).sum()) / resid_wt.shape[0]} ft.")

In [ ]:
print(f"Lower Aquifer Statistics\nMean Error: {resid_aq.mean()} ft.\nRMSE:       {np.sqrt((resid_aq**2).sum()) / resid_aq.shape[0]} ft.")

In [ ]:
print(f"Global Statistics\nMean Error: {resid_gb.mean()} ft.\nRMSE:       {np.sqrt((resid_gb**2).sum()) / resid_gb.shape[0]} ft.")

##### Plot the residuals

In [ ]:
xy = [(float(gwf.modelgrid.xcellcenters[i, j]), float(gwf.modelgrid.ycellcenters[i, j])) for i, j in obs_rc_locs]

In [ ]:
x, y = np.array(xy)[:, 0], np.array(xy)[:, 1]

In [ ]:
grid_x, grid_y = np.meshgrid(gwf.modelgrid.xycenters[0],
                             gwf.modelgrid.xycenters[1])

In [ ]:
# Linearly interpolate the data (x, y) on a grid defined by (xi, yi).
triang = tri.Triangulation(x, y)

In [ ]:
interpolator = tri.LinearTriInterpolator(triang, resid_wt)
grid_resid_wt = interpolator(grid_x, grid_y)

In [ ]:
interpolator = tri.LinearTriInterpolator(triang, resid_aq)
grid_resid_aq = interpolator(grid_x, grid_y)

In [ ]:
resid_levels = np.arange(-2, 2.25, .25)

In [ ]:
with flopy.plot.styles.USGSMap():
    fig, axs = plt.subplots(1, 2, figsize=(8, 5), sharey=True)
    fig.suptitle("Residuals")

    ax = axs[0]
    ax.set_xlim(0, 12500)
    ax.set_ylim(0, 20000)
    mm = flopy.plot.PlotMapView(model=gwf, ax=ax, extent=gwf.modelgrid.extent)
    mm.plot_array(lake_location, cmap="Blues_r", masked_values=[0])
    mm.plot_grid(lw=0.5, color="0.5")
    ax.scatter(x, y, s=3, c="black")
    for i, txt in enumerate(resid_wt):
        ax.annotate(f"{txt:.2f}", (x[i], y[i]))
    cs = ax.contour(grid_x, grid_y, grid_resid_wt, levels=resid_levels, linewidths=0.75, colors="red")
    plt.clabel(cs, inline=True, fontsize=8)
    ax.set_title("Water Table")

    ax = axs[1]
    ax.set_xlim(0, 12500)
    ax.set_ylim(0, 20000)
    mm = flopy.plot.PlotMapView(model=gwf, ax=ax, extent=gwf.modelgrid.extent)
    mm.plot_grid(lw=0.5, color="0.5")
    ax.scatter(x, y, s=3, c="black")
    for i, txt in enumerate(resid_aq):
        ax.annotate(f"{txt:.2f}", (x[i], y[i]), clip_on=False)
    cs = ax.contour(grid_x, grid_y, grid_resid_aq, levels=resid_levels, linewidths=0.75, colors="red")
    plt.clabel(cs, inline=True, fontsize=8)
    ax.set_title("Lower Aquifer")

**NOTE:** There is spatial bias in the simulated results (*i.e.*, residuals are positive in the Northeast and negative in the Southwest).

#### Case A

In [ ]:
with flopy.plot.styles.USGSMap():
    fig, axs = plt.subplots(1, 5, figsize=(9, 3), sharey=True)
    fig.suptitle("Heads - Case A")
    hds = gwf.output.head().get_data(totim=2.0)

    for idx in range(nlay):
        ax = axs[idx]
        mm = flopy.plot.PlotMapView(model=gwf, layer=idx, ax=ax)
        mm.plot_array(hds)
        cs = mm.contour_array(hds, colors="white", levels=levels)
        plt.clabel(cs, inline=True, fontsize=8)
        ax.set_title(f"Layer {idx + 1}")

In [ ]:
with flopy.plot.styles.USGSMap():
    fig, axs = plt.subplots(1, 5, figsize=(9, 3), sharey=True)
    fig.suptitle("Drawdown - Case A")
    ddn = gwf.output.head().get_data(totim=1.0) - gwf.output.head().get_data(totim=2.0) 

    ddn_max = ddn[:, 16, :].max()

    for idx in range(nlay):
        ax = axs[idx]
        mm = flopy.plot.PlotMapView(model=gwf, layer=idx, ax=ax)
        mm.plot_array(ddn)
        cs = mm.contour_array(ddn, colors="white", levels=levels)
        plt.clabel(cs, inline=True, fontsize=8)
        ax.set_title(f"Layer {idx + 1}")

In [ ]:
print(f"Maximum Drawdown: {ddn_max}")

In [ ]:
v = gwf.output.budget().get_data(text="riv", totim=2.0)[0]["q"]
print(f"Induced river infiltration: {np.all(v > 0)}\n{v}")

 #### Case B

In [ ]:
with flopy.plot.styles.USGSMap():
    fig, axs = plt.subplots(1, 5, figsize=(9, 3), sharey=True)
    fig.suptitle("Heads - Case B")
    hds = gwf.output.head().get_data(totim=3.0)

    for idx in range(nlay):
        ax = axs[idx]
        mm = flopy.plot.PlotMapView(model=gwf, layer=idx, ax=ax)
        mm.plot_array(hds)
        cs = mm.contour_array(hds, colors="white", levels=levels)
        plt.clabel(cs, inline=True, fontsize=8)
        ax.set_title(f"Layer {idx + 1}")

In [ ]:
with flopy.plot.styles.USGSMap():
    fig, axs = plt.subplots(1, 5, figsize=(9, 3), sharey=True)
    fig.suptitle("Drawdown - Case B")
    ddn = gwf.output.head().get_data(totim=1.0) - gwf.output.head().get_data(totim=3.0) 
    
    ddn_max = ddn[:, 16, :].max()
    
    for idx in range(nlay):
        ax = axs[idx]
        mm = flopy.plot.PlotMapView(model=gwf, layer=idx, ax=ax)
        mm.plot_array(ddn)
        cs = mm.contour_array(ddn, colors="white", levels=levels)
        plt.clabel(cs, inline=True, fontsize=8)
        ax.set_title(f"Layer {idx + 1}")

In [ ]:
print(f"Maximum Drawdown: {ddn_max}")

In [ ]:
v = gwf.output.budget().get_data(text="riv", totim=3.0)[0]["q"]
print(f"Induced river infiltration: {np.all(v > 0)}\n{v}")

#### Extra Run

In [ ]:
with flopy.plot.styles.USGSMap():
    fig, axs = plt.subplots(1, 5, figsize=(9, 3), sharey=True)
    fig.suptitle("Heads - Extra Run")
    hds = gwf.output.head().get_data(totim=4.0)

    for idx in range(nlay):
        ax = axs[idx]
        mm = flopy.plot.PlotMapView(model=gwf, layer=idx, ax=ax)
        mm.plot_array(hds)
        cs = mm.contour_array(hds, colors="white", levels=levels)
        plt.clabel(cs, inline=True, fontsize=8)
        ax.set_title(f"Layer {idx + 1}")

In [ ]:
with flopy.plot.styles.USGSMap():
    fig, axs = plt.subplots(1, 5, figsize=(9, 3), sharey=True)
    fig.suptitle("Drawdown - Extra Run")
    ddn = gwf.output.head().get_data(totim=1.0) - gwf.output.head().get_data(totim=4.0) 
    
    ddn_max = ddn[:, 16, :].max()

    for idx in range(nlay):
        ax = axs[idx]
        mm = flopy.plot.PlotMapView(model=gwf, layer=idx, ax=ax)
        mm.plot_array(ddn)
        cs = mm.contour_array(ddn, colors="white", levels=levels)
        plt.clabel(cs, inline=True, fontsize=8)
        ax.set_title(f"Layer {idx + 1}")

In [ ]:
print(f"Maximum Drawdown: {ddn_max}")

In [ ]:
v = gwf.output.budget().get_data(text="riv", totim=4.0)[0]["q"]
print(f"Induced river infiltration: {np.all(v > 0)}\n{v}")

### Streamflow results

In [ ]:
df = riv.output.obs().get_dataframe()
df["PF"] /= -86400
df["RIVFLOW"] /= -86400
df["TOTAL"] = df["PF"] + df["RIVFLOW"]
df

In [ ]:
Q = df["TOTAL"].values
Q

In [ ]:
with flopy.plot.styles.USGSPlot():
    fig, axs = plt.subplots(2, 1, figsize=(9, 3), sharex=True)

    fig.suptitle("Southern Boundary - Gage 1")

    ax = axs[0]
    ax.set_ylim(5, 12)
    ax.plot(df["totim"], Q, ls="-", marker="o", clip_on=False)
    ax.set_ylabel("River\nDischarge, cfs")

    ax = axs[1]
    ax.set_ylim(0, 50)
    ax.plot(df["totim"], -100. * (Q - Q[0]) / Q[0], ls="-", marker="o", clip_on=False)
    ax.set_ylabel("Reduction\n in River\nDischarge, %")
    ax.set_xlabel("Stress Period")
    ax.set_xticks([1, 2, 3, 4], ["Calibration", "Case A", "Case B", "Extra Run"])


In [ ]:
Q = df["PF"].values
Q

In [ ]:
with flopy.plot.styles.USGSPlot():
    fig, axs = plt.subplots(2, 1, figsize=(9, 3), sharex=True)

    fig.suptitle("Pollock's Ford - Gage 2")

    ax = axs[0]
    ax.set_ylim(1, 1.6)
    ax.plot(df["totim"], Q, ls="-", marker="o", clip_on=False)
    ax.set_ylabel("River\nDischarge, cfs")

    ax = axs[1]
    ax.set_ylim(0, 50)
    ax.plot(df["totim"], -100. * (Q - Q[0]) / Q[0], ls="-", marker="o", clip_on=False)
    ax.set_ylabel("Reduction\n in River\nDischarge, %")
    ax.set_xlabel("Stress Period")
    ax.set_xticks([1, 2, 3, 4], ["Calibration", "Case A", "Case B", "Extra Run"])


### Lake stage

In [ ]:
fpth = ws / "mv.gwf.scenario.csv"

In [ ]:
df = flopy.utils.Mf6Obs(fpth).get_dataframe()
df

In [ ]:
with flopy.plot.styles.USGSPlot():
    fig, ax = plt.subplots(1, 1, figsize=(9, 1.5))

    ax.plot(df["totim"], df["LAKE-N"], ls="-", marker="o",)
    ax.set_ylabel("Lake\nStage, ft")
    ax.set_xlabel("Stress Period")
    ax.set_xticks([1, 2, 3, 4], ["Calibration", "Case A", "Case B", "Extra Run"])
    ax.set_ylim(8, 12)
